In [6]:
using BenchmarkTools, StaticArrays, LinearAlgebra

In [24]:
A = rand(7, 7)
B = SMatrix{7, 7}(A);

In [32]:
@btime inv(A)
@btime inv(B)

  979.167 ns (8 allocations: 4.19 KiB)
  388.264 ns (1 allocation: 400 bytes)


7×7 SMatrix{7, 7, Float64, 49} with indices SOneTo(7)×SOneTo(7):
  0.235199  -3.09555     5.68592    …  -3.29009    -0.181692  -1.3493
  0.191677  -0.247511   -0.225837       1.08336    -0.531278   0.50763
 -0.317324  -2.58983     9.55294       -5.26092    -2.07201   -3.65397
 -0.882315   1.66107    -4.12158        2.62308     1.19137    2.5162
  0.554715  -1.39467     1.9912        -0.702875   -0.609882  -2.84823
  0.440958  -0.016649    0.0531497  …  -0.0400529   0.58262   -0.870545
  0.451084   4.68122   -11.3735         5.33155     1.62381    4.70843

In [31]:
991.6/389.365

2.5467106699369486